In [2]:
import numpy as np
import pandas as pd

In [3]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        a = self.lexicon * self.prior
        return rownorm(a)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        u = self.speaker().T
        return rownorm(u)

In [4]:
def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T

In [5]:
def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)

In [6]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
mustache,0.00,1.00,0.0


скалярную импликатуру вывести можно. основываясь на распределении вероятностей, можно сделать вывод: когда прагматический говорящий говорит о "человеке в шляпе", прагматический слушающий скорее выберет первый рисунок, чем второй. 

In [7]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.0,1.0,0.0,0.0
glasses,0.0,0.0,1.0,0.0
mustache,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.689655,0.310345,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0


при новом (не одинаковом) распределении априорных вероятностей прагматический слушающий все еще будет выбирать первый рисунок, но с меньше вероятностью, чем раньше.

In [8]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [0.0, 1.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,0.000000,1.0,0.0,0.0
glasses,1.000000,1.0,0.0,0.0
mustache,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
mustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.0,0.000000
glasses,0.600000,0.4,0.000000
mustache,0.333333,0.0,0.666667


если взять референциальную игру с семинара, где у двоих были очки и у двоих - усы. то можно увидеть, что при равной исходной вероятности, альфе = 1 и нулевой цене высказывания, прагматический слушающий будет чаще выбирать 1-й рисунок после фразы "человек с очками" и 3-й рисунок после фразы "человек с усами". 

In [9]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [0.0, 1.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.05, 0.475, 0.475], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,0.00,1.0,0.0,0.0
glasses,1.00,1.0,0.0,0.0
mustache,1.00,0.0,1.0,0.0
prior,0.05,0.475,0.475,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.095238,0.904762,0.000000
mustache,0.095238,0.000000,0.904762



Pragmatic speaker


,hat,glasses,mustache
r1,0.000,0.500,0.5
r2,0.525,0.475,0.0
r3,0.000,0.000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.512821,0.487179,0.000000
mustache,0.333333,0.000000,0.666667


если изменить исходные вероятности на P(r1) = 0.05, P(r2) = P(r3) = 0.475, то можно заметить, что прагматический слушающий будет выбирать 1-й рисунок почти так же часто, как и 2-й после фразы "человек с очками". 

In [10]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [0.0, 1.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 4.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,0.000000,1.0,0.0,0.0
glasses,1.000000,1.0,0.0,4.0
mustache,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
mustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,0.000000,0.982014,0.017986
r2,0.035337,0.964663,0.000000
r3,0.000000,0.000000,1.000000



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.504456,0.495544,0.000000
mustache,0.017668,0.000000,0.982332


если изменить цену высказывания "человек с очками" с 0.0 на 4.0, то можно увидеть, что 1-й рисунок в таком случае будут выбирать почти также часто, что и 2-й. в то время как, после высказывания "человек с усами" почти всегда будут выбирать 3-ю картинку вместо 1-й (что отличается от результатов игры при равных априорных вероятностях и нулевой цене высказывания у всех высказываний). соответственно, если поднять цену высказывания "человек с усами" до 4.0, а цену высказывания "человек с очками" оставить 0.0, получится обратная ситуация.  

In [12]:
if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [0.0, 1.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha = 4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,0.000000,1.0,0.0,0.0
glasses,1.000000,1.0,0.0,0.0
mustache,1.000000,0.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,4.000000,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
mustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,0.000000,0.500000,0.5
r2,0.941176,0.058824,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.894737,0.105263,0.000000
mustache,0.333333,0.000000,0.666667


если поменять параметр альфа с 1.0 на 4.0, то вероятность выбора 1-го рисунка при высказывании "человек с очками" увеличивается с 0.6 до 0.89. при этом, распределение вероятностей для высказывания "человек с усами" не меняется. 

теория рационального речевого акта может быть интегрирована в оценку навыков прагматического суждения у людей с афазией. так как при афазии некоторые пациенты могут затрудняться в распознавании скалярных импликатур в силу сложности обработки абстрактных понятий и более сложного синтаксиса. возможно, в таком случае, будет несколько смещено распределение априорных вероятностей и изменится цена высказывания. 
https://pure.ulster.ac.uk/ws/files/11639977/Scalar_implicatures_and_Presuppositions.pdf